In [1]:
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
import pandas_gbq
import glob

In [2]:
client = bigquery.Client()

In [4]:
query = """SELECT 
  ano,
  sigla_uf,
  id_municipio,

count(*)

FROM `basedosdados.br_me_rais.microdados_vinculos`
WHERE (natureza_juridica LIKE "1%" 
OR natureza_juridica IN ('2011', '2038'))
AND natureza_juridica != '1228'
AND cbo_2002 = "251605" 
AND vinculo_ativo_3112 = 1
and ano > 2017


GROUP BY 

ano,
sigla_uf,
id_municipio
"""

In [25]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,sigla_uf,id_municipio,f0_
0,2019,AC,1200179,3
1,2020,RR,1400175,5
2,2021,AP,1600709,4
3,2021,AP,1600253,4
4,2021,AC,1200302,3
...,...,...,...,...
18592,2020,SP,3542008,2
18593,2020,SP,3542503,3
18594,2020,SP,3542909,3
18595,2020,SP,3549508,2


In [26]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

In [27]:
df= df.rename(columns={'f0_':'quantidade_vinculos'})
df

,ano,sigla_uf,id_municipio,quantidade_vinculos
0,2019,AC,1200179,3
1,2020,RR,1400175,5
2,2021,AP,1600709,4
3,2021,AP,1600253,4
4,2021,AC,1200302,3
...,...,...,...,...
18592,2020,SP,3542008,2
18593,2020,SP,3542503,3
18594,2020,SP,3542909,3
18595,2020,SP,3549508,2


In [ ]:
##puxar pop e dar merge

In [15]:
pop = pd.read_csv("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\IBGE\\Censo_previa_pop.csv", decimal=',', sep=';')

In [28]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [29]:
df1

,ano,sigla_uf,id_municipio,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO
8866,2018,SP,3550308,2473,SP,São Paulo,12200180.0
8868,2021,SP,3550308,2353,SP,São Paulo,12200180.0
8865,2019,SP,3550308,2346,SP,São Paulo,12200180.0
8867,2020,SP,3550308,2268,SP,São Paulo,12200180.0
4035,2019,RJ,3304557,1879,RJ,Rio de Janeiro,6625849.0
...,...,...,...,...,...,...,...
3281,2020,RS,4319125,1,RS,São Martinho da Serra,2919.0
15546,2019,RS,4300661,1,RS,André da Rocha,1129.0
15547,2020,RS,4300661,1,RS,André da Rocha,1129.0
15548,2021,RS,4300661,1,RS,André da Rocha,1129.0


In [33]:
df1= df1.drop('UF', axis=1)


In [34]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [35]:
df1

,ano,sigla_uf,id_municipio,quantidade_vinculos,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
8866,2018,SP,3550308,2473,São Paulo,12200180.0,0.202702
8868,2021,SP,3550308,2353,São Paulo,12200180.0,0.192866
8865,2019,SP,3550308,2346,São Paulo,12200180.0,0.192292
8867,2020,SP,3550308,2268,São Paulo,12200180.0,0.185899
4035,2019,RJ,3304557,1879,Rio de Janeiro,6625849.0,0.283586
...,...,...,...,...,...,...,...
3281,2020,RS,4319125,1,São Martinho da Serra,2919.0,0.342583
15546,2019,RS,4300661,1,André da Rocha,1129.0,0.88574
15547,2020,RS,4300661,1,André da Rocha,1129.0,0.88574
15548,2021,RS,4300661,1,André da Rocha,1129.0,0.88574


In [36]:
df1 = df1.rename(columns={'NOME DO MUNICÍPIO':'nome_municipio',
                          'POPULAÇÃO':'populacao_domiciliada',})
df1

,ano,sigla_uf,id_municipio,quantidade_vinculos,nome_municipio,populacao_domiciliada,taxa
8866,2018,SP,3550308,2473,São Paulo,12200180.0,0.202702
8868,2021,SP,3550308,2353,São Paulo,12200180.0,0.192866
8865,2019,SP,3550308,2346,São Paulo,12200180.0,0.192292
8867,2020,SP,3550308,2268,São Paulo,12200180.0,0.185899
4035,2019,RJ,3304557,1879,Rio de Janeiro,6625849.0,0.283586
...,...,...,...,...,...,...,...
3281,2020,RS,4319125,1,São Martinho da Serra,2919.0,0.342583
15546,2019,RS,4300661,1,André da Rocha,1129.0,0.88574
15547,2020,RS,4300661,1,André da Rocha,1129.0,0.88574
15548,2021,RS,4300661,1,André da Rocha,1129.0,0.88574


In [38]:
df1= df1[['ano', 'sigla_uf', 'id_municipio', 'nome_municipio', 'quantidade_vinculos', 'populacao_domiciliada', 'taxa']]

In [39]:
df1

,ano,sigla_uf,id_municipio,nome_municipio,quantidade_vinculos,populacao_domiciliada,taxa
8866,2018,SP,3550308,São Paulo,2473,12200180.0,0.202702
8868,2021,SP,3550308,São Paulo,2353,12200180.0,0.192866
8865,2019,SP,3550308,São Paulo,2346,12200180.0,0.192292
8867,2020,SP,3550308,São Paulo,2268,12200180.0,0.185899
4035,2019,RJ,3304557,Rio de Janeiro,1879,6625849.0,0.283586
...,...,...,...,...,...,...,...
3281,2020,RS,4319125,São Martinho da Serra,1,2919.0,0.342583
15546,2019,RS,4300661,André da Rocha,1,1129.0,0.88574
15547,2020,RS,4300661,André da Rocha,1,1129.0,0.88574
15548,2021,RS,4300661,André da Rocha,1,1129.0,0.88574


In [40]:
df1.dtypes

ano                        Int64
sigla_uf                  object
id_municipio               int64
nome_municipio            object
quantidade_vinculos        Int64
populacao_domiciliada    float64
taxa                     Float64
dtype: object

Subindo para o GBQ

In [41]:
client = bigquery.Client()
dataset_ref = client.dataset('Datalake')

In [42]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos de assistentes sociais'),
 bigquery.SchemaField('taxa','FLOAT',description='Taxa de assistentes sociais por 1000 habitantes')
 ]

In [44]:
table_ref = dataset_ref.table('RAIS_vinculos_assistentes_sociais_mil_habitantes')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=60bb98c5-4c8a-47a5-8c5c-8851f29912a4>